In [1]:
### 사전준비
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from matplotlib import font_manager, rc
import platform
import json
import folium
if platform.system() == "Windows" :
    path = "c:/Windows/Fonts/Malgun.ttf"
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc("font", family = font_name)
elif platform.system() == "Drawin" :
    rc("font",family = "AppleGothic")
else :
    print("check your OS system...")
plt.rcParams["axes.unicode_minus"] = False
from bs4 import BeautifulSoup
import requests
from openpyxl.workbook import Workbook
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import random

In [2]:
path = "../01_raw데이터/2019년_원자료.csv"
df = pd.read_csv(path)

In [4]:
# 변수명 읽어오기(조장님)

df = pd.read_excel('./데이터_변수정리_찐최종본.xlsx',
                    header =1,
                    usecols=[1,6,14],
                    sheet_name=0,
                    names = ['val1', 'val2', 'val3'])

val_lis = list(df['val1'].dropna(axis = 0))

val_lis2 = []
for val in val_lis:
    if 'yq9a' in val:
        temp = ['yq9a{}'.format(str(i).zfill(2)) for i in range(1, 18)]
        
    elif '~' in val:
        text = re.compile('[0-9]~').search(val).start()
        i_start = re.compile('[0-9]{0,3}$').search(val).start()
        i_end = re.compile('[0-9]{0,3}$').search(val).end()
        ran = val[int(i_start):int(i_end)]
        for i in range(1,int(ran)+1):
            val_lis2.append('{}{}'.format(val[:text],i))
            
    else:
        val_lis2.append(val)

file_dir = '../01_raw데이터/'
df19 = pd.read_csv(file_dir+'2019년_원자료.csv')
df19 = df19.loc[:,val_lis2]
df19

,Q1,Q2A1,Q2A2,Q2A3,Q2A4,Q2A5,Q5_2A1,Q5_2A2,Q5_2A3,Q9_2A1,...,Q15_2A2,Q15_2A3,Q15_2A4,Q15_2A5,Q15_2A6,Q15_2A7,Q15_2A8,Q15_2A10,Q15_2A12,Q16
0,2.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,6.0,1.0,...,4.0,4.0,5.0,5.0,3.0,5.0,4.0,4.0,3.0,5.0
1,1.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,1.0,...,3.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,3.0,5.0
2,3.0,3.0,NaN,NaN,NaN,NaN,2.0,5.0,NaN,1.0,...,4.0,3.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,3.0
3,10.0,1.0,NaN,NaN,NaN,NaN,2.0,5.0,NaN,1.0,...,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,0.0,2.0
4,1.0,3.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,1.0,...,4.0,5.0,5.0,5.0,3.0,4.0,4.0,2.0,3.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16071,2.0,1.0,NaN,NaN,NaN,NaN,10.0,4.0,11.0,5.0,...,5.0,5.0,4.0,4.0,5.0,3.0,5.0,3.0,4.0,5.0
16072,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,...,5.0,5.0,5.0,0.0,0.0,5.0,5.0,5.0,0.0,5.0
16073,1.0,1.0,NaN,NaN,NaN,NaN,9.0,2.0,1.0,10.0,...,4.0,4.0,5.0,4.0,5.0,4.0,5.0,5.0,0.0,5.0
16074,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,4.0,4.0,2.0,0.0,3.0,3.0,5.0,2.0,2.0,5.0


In [5]:
### 응답항목들을 변수명에 맞추어 정렬
val = 'Q2A1~Q2A5'
df19['q2a1'] = np.nan
df19['q2a2'] = np.nan
df19['q2a3'] = np.nan
df19['q2a4'] = np.nan
df19['q2a5'] = np.nan

text = re.compile('[0-9]~').search(val).start()
i_start = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).start()+1
i_end = re.compile('[a-zA-Z][0-9]{0,3}$').search(val).end()
ran = val[int(i_start):int(i_end)]

for i in range(1,int(ran)+1):
    name = '{}{}'.format(val[:text],i)

    if 'Q2A' in name:
        for j in range(0,len(df19)):
            if df19[name][j] == 1:
                df19['q2a1'][j] = 1
            elif df19[name][j] == 2:
                df19['q2a2'][j] = 2
            elif df19[name][j] == 3:
                df19['q2a3'][j] = 3
            elif df19[name][j] == 4:
                df19['q2a4'][j] = 4
            elif df19[name][j] == 5:
                df19['q2a5'][j] = 5

In [6]:
# 필요없는 컬럼 제거, 정렬한 컬럼명 재설정
df19 = df19.drop(['Q2A1','Q2A2','Q2A3','Q2A4','Q2A5','Q14_2_8', 'Q14_2_9'], axis=1)
df19.rename(columns={'q2a1':'Q2A1','q2a2':'Q2A2','q2a3':'Q2A3','q2a4':'Q2A4','q2a5':'Q2A5'},inplace=True)

In [7]:
# 저장
df19.to_csv("./2019년_변수정리.csv",index=False)

In [8]:
# 컬럼 확인
df19.columns

Index(['Q1', 'Q5_2A1', 'Q5_2A2', 'Q5_2A3', 'Q9_2A1', 'Q9_2A2', 'Q9_2A3',
       'Q10_1B1', 'Q10_1B2', 'Q10_1B3', 'Q10_1B4', 'Q10_1B5', 'Q10_1B6',
       'Q10_1B7', 'Q10_1B8', 'Q10_1B9', 'Q10_1B10', 'Q10_1B11', 'Q10_1B12',
       'Q10_1B13', 'Q10_1B14', 'Q10_1B15', 'Q10_1B16', 'Q10_1B17', 'Q10_2A1',
       'Q10_2A2', 'Q10_2A3', 'Q14_1T', 'CQ14_2_1', 'CQ14_2_2', 'CQ14_2_3',
       'CQ14_2_4', 'CQ14_2_5', 'CQ14_2_6', 'CQ14_2_7', 'Q14_2_10', 'ID',
       'D_MON', 'D_COU1', 'D_GEN', 'D_AGE', 'D_Q5_1', 'Q15_2A1', 'Q15_2A2',
       'Q15_2A3', 'Q15_2A4', 'Q15_2A5', 'Q15_2A6', 'Q15_2A7', 'Q15_2A8',
       'Q15_2A10', 'Q15_2A12', 'Q16', 'Q2A1', 'Q2A2', 'Q2A3', 'Q2A4', 'Q2A5'],
      dtype='object')

In [10]:
# 컬럼 개수 확인
len(['Q1', 'Q5_2A1', 'Q5_2A2', 'Q5_2A3', 'Q9_2A1', 'Q9_2A2', 'Q9_2A3',
       'Q10_1B1', 'Q10_1B2', 'Q10_1B3', 'Q10_1B4', 'Q10_1B5', 'Q10_1B6',
       'Q10_1B7', 'Q10_1B8', 'Q10_1B9', 'Q10_1B10', 'Q10_1B11', 'Q10_1B12',
       'Q10_1B13', 'Q10_1B14', 'Q10_1B15', 'Q10_1B16', 'Q10_1B17', 'Q10_2A1',
       'Q10_2A2', 'Q10_2A3', 'Q14_1T', 'CQ14_2_1', 'CQ14_2_2', 'CQ14_2_3',
       'CQ14_2_4', 'CQ14_2_5', 'CQ14_2_6', 'CQ14_2_7', 'Q14_2_10', 'ID',
       'D_MON', 'D_COU1', 'D_GEN', 'D_AGE', 'D_Q5_1', 'Q15_2A1', 'Q15_2A2',
       'Q15_2A3', 'Q15_2A4', 'Q15_2A5', 'Q15_2A6', 'Q15_2A7', 'Q15_2A8',
       'Q15_2A10', 'Q15_2A12', 'Q16', 'Q2A1', 'Q2A2', 'Q2A3', 'Q2A4', 'Q2A5'])

58